# RecallioMemory + LangChain Integration Demo
A minimal notebook to show drop-in usage of RecallioMemory in LangChain (with scoped writes and recall).

In [ ]:
%pip install recallio langchain langchain-recallio openai

## Setup: API Keys & Imports

In [ ]:
from langchain_recallio.memory import RecallioMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import os

# Set your keys here or use environment variables
RECALLIO_API_KEY = os.getenv("RECALLIO_API_KEY", "YOUR_RECALLIO_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY")

## Initialize RecallioMemory

In [ ]:
memory = RecallioMemory(
    project_id="project_abc",
    api_key=RECALLIO_API_KEY,
    session_id="demo-session-001",
    user_id="demo-user-42",
    default_tags=["test", "langchain"],
    return_messages=True,
)

## Build a LangChain ConversationChain with RecallioMemory

In [ ]:
# You can swap in any supported LLM here
llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "The following is a friendly conversation between a human and an AI. "
            "The AI is talkative and provides lots of specific details from its context. "
            "If the AI does not know the answer to a question, it truthfully says it does not know.",
        ),
        ("placeholder", "{history}"),  # RecallioMemory will fill this slot
        ("human", "{input}"),
    ]
)

# LCEL chain that returns an AIMessage
base_chain = prompt | llm


# Create a stateful chain using RecallioMemory
def chat_with_memory(user_input: str):
    # Load conversation history from memory
    memory_vars = memory.load_memory_variables({"input": user_input})

    # Run the chain with history and user input
    response = base_chain.invoke(
        {"input": user_input, "history": memory_vars.get("history", "")}
    )

    # Save the conversation to memory
    memory.save_context({"input": user_input}, {"output": response.content})

    return response

## Example: Chat with Memory

In [ ]:
# First user message – note the AI remembers the name
resp1 = chat_with_memory("Hi! My name is Guillaume. Remember that.")
print("Bot:", resp1.content)

Bot: Hello Guillaume! It's nice to meet you. How can I assist you today?


In [ ]:
# Second user message – AI should recall the name from memory
resp2 = chat_with_memory("What is my name?")
print("Bot:", resp2.content)

Bot: Your name is Guillaume.


## See What Is Stored in Recallio
This is for debugging/demo only; in production, you wouldn't do this on every run.

In [ ]:
print("Current memory variables:", memory.load_memory_variables({}))

Current memory variables: {'history': [HumanMessage(content='Name is Guillaume', additional_kwargs={}, response_metadata={})]}


## Clear Memory (Optional Cleanup - Requires Manager level Key)

In [ ]:
# memory.clear()
# print("Memory cleared.")